<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right" />

<h1 align=left><font size=8><b>Inteligência Artificial</b></font></h1>
<h1 align=left><font size=6><b>Deep Learning</b></font></h1>

# Atividade: T6 - Atividade de aprofundamento

Nome: Bruno Rebocho de Toledo

Turma: 01B

Matrícula: 92316328

---
## Introdução

Este notebook tem como objetivo explorar as capacidades de classificação de redes neurais com PyTorch.

---
## Dataset "Predict students' dropout and academic success"[1]

O conjunto de dados escolhido, criado a partir de uma instituição de ensino superior (adquirido de várias bases de dados independentes), descreve estudantes matriculados em diferentes graduações, como agronomia, design, educação, enfermagem, jornalismo, gestão, serviço social e tecnologias. O conjunto de dados inclui informações conhecidas no momento da matrícula do estudante (trajetória acadêmica, demografia e fatores socioeconômicos) e o desempenho acadêmico dos alunos ao final do primeiro e segundo semestres. Os dados são usados para construir modelos de classificação para prever a evasão e o sucesso acadêmico.

---
## O Problema

O problema é formulado como uma tarefa de classificação binária indicando a conclusão ou desistência do curso, na qual existe um forte desequilíbrio em favor de uma das classes originais.

## Uso de GPU

Iniciamos configurando nosso ambiente de execução para utilizar a GPU, caso disponível. As operações a serem executadas no treinamento deste modelo tendem a serem feitas de maneira muito mais rápida e eficiente em GPUs, devido à natureza da arquitetura deste tipo de chip e as estruturas dos dados aos quais iremos operar.

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

ModuleNotFoundError: No module named 'torch'

---
## Explorando os Dados

Inicialmente, começamos explorando os dados ao importar as bibliotecas `numpy` e `pandas`. Utilizamos o `pandas` para ler o arquivo 'data.csv', especificando o ponto e vírgula como delimitador.

Após a leitura do arquivo, exibimos as primeiras cinco linhas do DataFrame `df` com a função `head()`. Esta visualização inicial nos fornece uma visão rápida de como os dados estão estruturados, incluindo nomes de colunas e tipos de dados, o que é crucial para o planejamento das etapas seguintes de análise.

Em seguida, imprimimos a dimensão do DataFrame com `df.shape`, revelando o número total de linhas e colunas. Esta informação é fundamental para entender o volume e a complexidade dos dados com os quais estamos trabalhando, orientando as decisões sobre o processamento e a análise subsequente.

Por vim, verificamos a presença de nulos ou valores ausentes, o que não é o caso deste nosso conjunto de dados.

In [ ]:
import numpy as np
import pandas as pd

!wget -O data.csv 'https://drive.google.com/uc?export=download&id=1hnHg7IN3qTPnIui2XrkT5hUd9XFH8Ecn'

df = pd.read_csv('data.csv', delimiter=';')

display(df.head())

print(df.shape)

In [ ]:
df.isnull().sum()

#### Avaliando a Distribuição da Coluna Alvo

Prosseguindo com a exploração dos dados, focamos na coluna 'Target', que desempenha um papel crucial na nossa análise. Para entender melhor a distribuição dos valores nesta coluna, utilizamos o método `value_counts()` do Pandas. Este método nos retorna rapidamente um panorama da frequência de cada valor único na coluna 'Target'.

Ao realizar esta operação, estamos buscando informações sobre a distribuição das classes ou categorias representadas na coluna 'Target'. Esta informação é vital, pois uma distribuição desigual das classes pode influenciar significativamente o desempenho do modelo.

Assim, ao imprimir o resultado de `value_counts()`, estabelecemos uma base para compreender como os dados estão estruturados em relação à variável que pretendemos prever.

Como esperado, há um forte desbalanceamento entre os valores.

In [ ]:
print(df['Target'].value_counts())

---
## Preparação dos Dados para Modelagem

Como estamos trabalhando com PyTorch, é boa prática criar uma classe customizada que extende a classe Dataset. Definimos um construtor com os parâmetros `X` e `y` a serem informados externamente. Esta abordagem foi escolhida pois facilita a normalização dos dados fora do contexto desta classe e simplifica sua reutilização.

Os métodos `__len__` e `__getitem__` também foram implementados pois são necessários para trabalhar em conjunto com o `DataLoader`.

Com o objetivo de adaptar nosso conjunto de dados para um problema de classificação binária, fizemos ajustes focando nas categorias 'Graduate' e 'Dropout'. Criamos a nova coluna 'Dropout' no DataFrame, codificando 'Graduate' como 0 e 'Dropout' como 1, que será nossa variável dependente. Eliminamos a categoria 'Enrolled', o que também contribui para reduzir o desbalanceamento inicial das classes.

Com a nova organização dos dados, removemos a coluna 'Target', pois ela já foi codificada em 'Dropout'. Em seguida, separamos o DataFrame em variáveis independentes `X` e a variável dependente `y`, que agora reflete com precisão o nosso problema binário de prever se um aluno irá se formar ou desistir.

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, X, y):
        self.X = X
        self.y = y

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

df = df[df['Target'].isin(['Graduate', 'Dropout'])]
df['Dropout'] = [0 if typ == 'Graduate' else 1 for typ in df['Target']]
df.drop(['Target'], axis=1, inplace=True)

X = df.values[:, :-1].astype('float32')
y = LabelEncoder().fit_transform(df.values[:, -1]).astype('float32').reshape((-1, 1))

---
## Separando os Dados em Treinamento e Teste

Agora avançamos para  a divisão do conjunto de dados em partes para treinamento e teste. Ao definirmos o parâmetro `test_size` como 0.3, estamos especificando que 30% do conjunto de dados será reservado para teste e 70% para treinar o modelo, conforme solicitado.

Após a aplicação da `train_test_split`, obtemos quatro conjuntos de dados: `X_train` e `y_train`, utilizados para treinamento, e `X_test` e `y_test`, que servirão para testar o modelo após o treinamento.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)

---
## Normalizando os Dados

Iniciamos este processo utilizando o método `describe()` do DataFrame, que fornece um resumo estatístico, como média, desvio padrão, valores mínimos e máximos. Esta visão permite identificar a necessidade de normalização.

In [ ]:
df.describe()

---
## Redução de Dimensionalidade e Pipeline de Pré-processamento

Neste estágio, aprimoramos o pré-processamento de dados incorporando a Análise de Componentes Principais (PCA). Começamos inicialmente com a importação e configuração do PCA da biblioteca `sklearn.decomposition`, definindo-o para reter 80% da variância dos dados.

No conjunto de treinamento (`X_train`), usamos o método `fit_transform` para ajustar o pipeline aos dados e, simultaneamente, transformá-los. O resultado dessa transformação é armazenado em `X_train_pca`. Para o conjunto de teste (`X_test`), empregamos o método `transform` para aplicar as mesmas transformações com os parâmetros ajustados durante o treinamento. Isso resulta em `X_test_pca`.

Em seguida, criamos duas instâncias de `CSVDataset`, uma para o conjunto de treinamento e outra para o conjunto de testes.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()

pca = PCA(n_components=0.8)

pipeline = make_pipeline(scaler, pca)

X_train_pca = pipeline.fit_transform(X_train)

X_test_pca = pipeline.transform(X_test)

# Criar datasets PyTorch
train_dataset = CSVDataset(X_train_pca, y_train)
test_dataset = CSVDataset(X_test_pca, y_test)

In [ ]:
train_dataset.X, test_dataset.X

## Aplicação do DataLoader

A utilização de um `DataLoader` é boa prática para o treinamento de modelos de redes neurais com PyTorch. Ele encapsula o gerenciamento dos dados e permite customizações sob demanda.

Definimos um `DataLoader` para o nosso conjunto de dados de testes. O tamanho do batch foi definido em 32. Decidimos por ativar o embaralhamento (`shuffle=True`) pois este pode ser benéfico ao treinamento, evitando que o modelo memorize a ordem dos dados. Se os dados não forem embaralhados, o modelo pode 'aprender' a predizer o próximo batch baseado na ordem dos dados, ao invés de fato aprender com os padrões presentes nestes dados [3].



In [ ]:
from torch.utils.data import DataLoader

train = DataLoader(train_dataset, batch_size=32, shuffle=True)

---
## Definindo o Modelo de Rede Neural

Determinamos o número de características de entrada (`n_inputs`) com base na forma do conjunto de dados de teste processado pelo PCA. Esse número guia a configuração da primeira camada da rede.

O modelo é construído como uma sequência de camadas, cada uma com a função de ativação `relu`, exceto a última camada que utiliza `sigmoid`. A primeira camada recebe `shape` neurônios, correspondendo ao número de características de entrada. A segunda camada tem metade do número de neurônios da primeira, seguindo uma estrutura que gradualmente condensa a informação. A última camada, com um único neurônio e ativação `sigmoid`, é adequada para uma tarefa de classificação binária, produzindo uma saída entre 0 e 1.

A função de perda escolhida foi a `BCELoss` (Binary Cross Entropy Loss) e o optimizador `Adam`, ambos adequados para a proposta de classificação binária.

In [ ]:
from torch.optim import SGD, Adam
import torch.nn as nn

class MLP(nn.Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()

        self.hidden1 = nn.Linear(n_inputs, 8)
        self.act1 = nn.ReLU()

        self.hidden2 = nn.Linear(8, 4)
        self.act2 = nn.ReLU()

        self.hidden3 = nn.Linear(4, 1)
        self.act3 = nn.Sigmoid()

    # forward
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)

        X = self.hidden2(X)
        X = self.act2(X)

        X = self.hidden3(X)
        X = self.act3(X)

        return X

model = MLP(train_dataset.X.shape[1]).to(device)
loss_fn = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=0.001)

print(model)

## Treinamento do Modelo

Configuramos um total de 10 iterações no treinamento do modelo. Este valor foi definido empiricamente.

Ao final do treinamento, o histórico é armazenado em uma variável acessória. Este histórico contém informações valiosas sobre a evolução da função de perda ao longo das épocas, permitindo a análise posterior do desempenho do modelo.

In [ ]:
import numpy as np

EPOCHS = 10
loss_list = np.zeros((EPOCHS,))

for epoch in range(EPOCHS):
    loss_batch = 0

    for x_batch, y_batch in train:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # compute the model output
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        # Zero gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_batch = loss_batch + loss.item()

    loss_list[epoch] = loss_batch / len(x_batch)

---
## Curva de Aprendizado e Avaliação do Desempenho do Modelo

Depois o treinamento, exibidos o histórico em um gráfico para análise.

O gráfico apresenta a evolução da perda ao longo das épocas. Observamos que a perda no conjunto de validação diminui de forma consistente ao longo do tempo, o que indica que o modelo está aprendendo efetivamente e melhorando a sua capacidade de generalização.

In [ ]:
import matplotlib.pyplot as plt

def plot_history(loss_list):

  plt.plot(loss_list)
  plt.ylabel("Train Loss")
  plt.xlabel("Epoch")
  plt.show()

plot_history(loss_list)

---
## Avaliação Final do Modelo

Concluído o treinamento da rede neural, procedemos à sua avaliação usando o conjunto de dados `test_dataset`. Note que o `DataLoader` está sendo utilizado e o parâmetro `batch_size=len(test_dataset)` indica que todo o conjunto de teste está sendo retornado para as variáveis `xx_test` e `yy_test`.

Em seguida, invocamos o modelo, passando o conjunto de teste `xx_test`, não sem antes enviá-lo para o contexto correto de execução. Em seguida, o retorno da predição é arredondado ( `round()`) e armazenado na variável `yy_pred`.

Por fim, calculamos a acurácia geral do modelo, que nos informa uma proporção de previsões corretas sobre o total de previsões. Com uma acurácia de aproximadamente 88%, podemos concluir que o modelo é bastante eficaz.

In [ ]:
from sklearn.metrics import accuracy_score

xx_test, yy_test = next(iter(DataLoader(test_dataset, batch_size=len(test_dataset))))

yy_pred = model(xx_test.to(device)).round()

print('Acuracidade: ', accuracy_score(yy_test.detach().cpu().numpy(), yy_pred.detach().cpu().numpy()))

## Adicionando a Regularização L2

Apesar da boa eficácia do modelo, em torno de 88%, ainda é possível explorar outras técnicas para obter melhores resultados. Neste exemplo, utilizaremos a Regularização L2, disponibilizada pelo parâmetro `weight_decay` da função de otimização. O valor `0.001` foi definido empiricamente.

Em seguida, treinamentos novamente o modelo com os mesmos parâmetros do treinamento anterior e avaliamos os resultados.

In [ ]:
model = MLP(train_dataset.X.shape[1]).to(device)
loss_fn = nn.BCELoss()

# weight_decay (float, optional) – weight decay (L2 penalty) (default: 0)
optimizer = Adam(model.parameters(),  weight_decay=0.001, lr=0.001)

print(model)

In [ ]:
import numpy as np

EPOCHS = 10
loss_list = np.zeros((EPOCHS,))

for epoch in range(EPOCHS):
    loss_batch = 0

    for x_batch, y_batch in train:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # compute the model output
        y_pred = model(x_batch)
        loss = loss_fn(y_pred, y_batch)

        # Zero gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_batch = loss_batch + loss.item()

    loss_list[epoch] = loss_batch / len(x_batch)

In [ ]:
plot_history(loss_list)

In [ ]:
test = DataLoader(test_dataset, batch_size=len(test_dataset))
xx_test, yy_test = next(iter(test))
yy_pred = model(xx_test.to(device)).round()

A acurácia em torno de 88% demonstra que a técnica de Regularização L2 não foi eficaz em aprimorar a acuracidade do modelo.

In [ ]:
print(accuracy_score(yy_test.detach().cpu().numpy(), yy_pred.detach().cpu().numpy()))

---
## Conclusões

Ao finalizar a análise e modelagem do conjunto de dados, concluímos que as redes neurais, com o uso do PyTorch, demonstraram ser ferramentas eficazes para prever a evasão e o sucesso acadêmico de estudantes.

A aplicação de técnicas como normalização e PCA e contribuíram significativamente para a eficácia do modelo.

A precisão alcançada, em torno de 88%, sugere que o modelo pode ser uma ferramenta valiosa para identificar estudantes em risco, permitindo intervenções proativas para melhorar a retenção e o sucesso acadêmico.

---
## Referências

1. University of California, Irvine. Predict Student's Dropout and Academic Success. UCI Machine Learning Repository. Disponível em: https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success. Acesso em: 23 nov.2023.

2. Pierian Training. Machine Learning in Python: Principal Component Analysis (PCA). Disponível em: https://pieriantraining.com/machine-learning-in-python-principal-component-analysis-pca/. Acesso em: 23 nov.2023.

3. Tantai, Hengtao. “Improving Control and Reproducibility of PyTorch DataLoader with Sampler Instead of Shuffle Argument.” Medium, 22 fev. 2023. Disponível em: https://medium.com/@zergtant/improving-control-and-reproducibility-of-pytorch-dataloader-with-sampler-instead-of-shuffle-7f795490256e. Acesso em: 25 nov. 2023.

4. PyTorch. torch.optim.Adam. Disponível em: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html. Acesso em: 25 nov.2023.